In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import Imputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

In [2]:
df=pd.read_csv('train_u6lujuX_CVtuZ9i.csv')
df_test=pd.read_csv('test_Y3wMUE5_7gLdaTN.csv')
print(df.head())
#df=df_o.drop('Loan_ID',axis=1)

    Loan_ID Gender Married Dependents     Education Self_Employed  \
0  LP001002   Male      No          0      Graduate            No   
1  LP001003   Male     Yes          1      Graduate            No   
2  LP001005   Male     Yes          0      Graduate           Yes   
3  LP001006   Male     Yes          0  Not Graduate            No   
4  LP001008   Male      No          0      Graduate            No   

   ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
0             5849                0.0         NaN             360.0   
1             4583             1508.0       128.0             360.0   
2             3000                0.0        66.0             360.0   
3             2583             2358.0       120.0             360.0   
4             6000                0.0       141.0             360.0   

   Credit_History Property_Area Loan_Status  
0             1.0         Urban           Y  
1             1.0         Rural           N  
2             1.0   

In [3]:
df.Gender.replace(' ',np.nan,inplace=True)
df.Married.replace(' ',np.nan,inplace=True)
df.Dependents.replace(' ',np.nan,inplace=True)
df.Self_Employed.replace(' ',np.nan,inplace=True)
df.Married.replace(' ',np.nan,inplace=True)
df.Loan_Amount_Term.replace(' ',np.nan,inplace=True)
df.Credit_History.replace(' ',np.nan,inplace=True)
df.LoanAmount = df.LoanAmount.fillna(df.LoanAmount.mean())
df.Self_Employed = df.Self_Employed.fillna('No')
df.Gender = df.Gender.fillna('Male')
df.Married = df.Married.fillna('Yes')
df.Loan_Status = df.Loan_Status.fillna('1')
df.Dependents.replace('3+','3',inplace=True)
df.Dependents=df.Dependents.fillna('0')
df.Credit_History = df.Credit_History.fillna('1.0')
df.Loan_Amount_Term=df.Loan_Amount_Term.fillna(df.Loan_Amount_Term.mean())
df_test.Dependents.replace('3+','3',inplace=True)
print(df.info())
df.isnull().any()
df_test.Gender.replace(' ',np.nan,inplace=True)
df_test.Gender = df_test.Gender.fillna('Male')
df_test.Dependents.replace(' ',np.nan,inplace=True)
df_test.Dependents=df_test.Dependents.fillna('0')
df_test.Self_Employed.replace(' ',np.nan,inplace=True)
df_test.Self_Employed = df_test.Self_Employed.fillna('No')
df.Loan_Amount_Term.replace(' ',np.nan,inplace=True)
df_test.Loan_Amount_Term=df_test.Loan_Amount_Term.fillna(df_test.Loan_Amount_Term.mean())
df_test.LoanAmount.replace(' ',np.nan,inplace=True)
df_test.LoanAmount = df_test.LoanAmount.fillna(df_test.LoanAmount.mean())
df_test.Credit_History.replace(' ',np.nan,inplace=True)
df_test.Credit_History = df_test.Credit_History.fillna('1.0')
print(df_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
Loan_ID              614 non-null object
Gender               614 non-null object
Married              614 non-null object
Dependents           614 non-null object
Education            614 non-null object
Self_Employed        614 non-null object
ApplicantIncome      614 non-null int64
CoapplicantIncome    614 non-null float64
LoanAmount           614 non-null float64
Loan_Amount_Term     614 non-null float64
Credit_History       614 non-null object
Property_Area        614 non-null object
Loan_Status          614 non-null object
dtypes: float64(3), int64(1), object(9)
memory usage: 62.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367 entries, 0 to 366
Data columns (total 12 columns):
Loan_ID              367 non-null object
Gender               367 non-null object
Married              367 non-null object
Dependents           367 non-null object
Education            36

In [4]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df['Gender']=le.fit_transform(df['Gender'])
df['Married']=le.fit_transform(df['Married'])
df['Education']=le.fit_transform(df['Education'])
df['Self_Employed']=le.fit_transform(df['Self_Employed'])
df['Property_Area']=le.fit_transform(df['Property_Area'])
df_test['Gender']=le.fit_transform(df_test['Gender'])
df_test['Education']=le.fit_transform(df_test['Education'])
df_test['Self_Employed']=le.fit_transform(df_test['Self_Employed'])
df_test['Property_Area']=le.fit_transform(df_test['Property_Area'])
df_test['Married']=le.fit_transform(df_test['Married'])
df_test['Loan_Status']=0
print(df.tail())
print(df_test.tail())
df['Loan_Status']=le.fit_transform(df['Loan_Status'])

      Loan_ID  Gender  Married Dependents  Education  Self_Employed  \
609  LP002978       0        0          0          0              0   
610  LP002979       1        1          3          0              0   
611  LP002983       1        1          1          0              0   
612  LP002984       1        1          2          0              0   
613  LP002990       0        0          0          0              1   

     ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
609             2900                0.0        71.0             360.0   
610             4106                0.0        40.0             180.0   
611             8072              240.0       253.0             360.0   
612             7583                0.0       187.0             360.0   
613             4583                0.0       133.0             360.0   

    Credit_History  Property_Area Loan_Status  
609              1              0           Y  
610              1              0     

In [5]:
#imp=Imputer(missing_values='NaN',strategy='most_frequent',axis=0)
steps=[('scaler',StandardScaler()),('knn',KNeighborsClassifier())]
pipeline=Pipeline(steps)
param_grid={'knn__n_neighbors':np.arange(1,50)}
pipeline_cv=GridSearchCV(pipeline,param_grid,cv=10)
y=df['Loan_Status']
X=df.drop(['Loan_Status','Loan_ID'],axis=1)
y_test=df['Loan_Status']
X_test=df_test.drop(['Loan_Status','Loan_ID'],axis=1)

In [13]:
print(X.head())

   Gender  Married Dependents  Education  Self_Employed  ApplicantIncome  \
0       1        0          0          0              0             5849   
1       1        1          1          0              0             4583   
2       1        1          0          0              1             3000   
3       1        1          0          1              0             2583   
4       1        0          0          0              0             6000   

   CoapplicantIncome  LoanAmount  Loan_Amount_Term Credit_History  \
0                0.0  146.412162             360.0              1   
1             1508.0  128.000000             360.0              1   
2                0.0   66.000000             360.0              1   
3             2358.0  120.000000             360.0              1   
4                0.0  141.000000             360.0              1   

   Property_Area  
0              2  
1              0  
2              2  
3              2  
4              2  


In [124]:
#X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=21)

In [6]:
pipeline_cv.fit(X,y)
y_pred=pipeline_cv.predict(X_test)
print(y_pred)
#le.fit(df['Loan_Status'])
#y_pred=le.inverse_transform(y_pred)
#print(y_pred)      

[1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 0 1 1 0 0 1 0 1 1 1 1
 1 1 1 1 1 1 0 1 0 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1
 1 1 1 1 1 1 0 0 0 1 1 1 0 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 0
 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 0 0 1 0 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 0 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1
 1 1 0 1 1 1 1 0 1 1 1 1 1 0 0 1 1 0 1 0 1 0 1 0 1 1 1 1 0 1 1 1 1 0 1 1 1
 1 1 1 1 1 1 1 0 1 0 1 1 1 1 0 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1
 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1]


In [7]:
print(pipeline_cv.best_params_)
print(pipeline_cv.best_score_)

{'knn__n_neighbors': 11}
0.809446254072


In [9]:
y_test=y_pred
col={'Loan_ID':df_test['Loan_ID'],'Loan_Status':y_test}
f=pd.DataFrame(col,index=np.arange(367))
print(f.head())
le.fit(['Y','N'])
f['Loan_Status']=le.inverse_transform(f['Loan_Status'])
print(f)
f.to_csv('sample_submission.csv',index=False)
print('Hey')

    Loan_ID  Loan_Status
0  LP001015            1
1  LP001022            1
2  LP001031            1
3  LP001035            1
4  LP001051            1
      Loan_ID Loan_Status
0    LP001015           Y
1    LP001022           Y
2    LP001031           Y
3    LP001035           Y
4    LP001051           Y
5    LP001054           Y
6    LP001055           Y
7    LP001056           N
8    LP001059           Y
9    LP001067           Y
10   LP001078           Y
11   LP001082           Y
12   LP001083           Y
13   LP001094           N
14   LP001096           Y
15   LP001099           Y
16   LP001105           Y
17   LP001107           Y
18   LP001108           Y
19   LP001115           Y
20   LP001121           Y
21   LP001124           Y
22   LP001128           Y
23   LP001135           Y
24   LP001149           Y
25   LP001153           N
26   LP001163           Y
27   LP001169           N
28   LP001174           Y
29   LP001176           Y
..        ...         ...
337  LP002856     